# Predicting Utilization with Logistic Regression
In this notebook, we will run queries against the data warehouse to create a dataset.  We will then use this dataset to build a model to predict the risk of hospitalization (admission or ER visit) in the next 6 months.

We will use logistic regression and its variants as our main modeling tool in this notebook.  Through this, we will see some of the strengths and weaknesses of logistic regression.

### Standard Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
#import xgboost as xgb
#import ml_insights as mli

pd.set_option("display.max_rows",9999)
pd.set_option("display.max_columns",9999)

### Run queries to get data

In [ ]:
import cloverdb

conn_str = cloverdb.connection_str_for_tunnel('stg-dwh-db')
#conn_str = cloverdb.connection_str_for_tunnel('prod-dwh-db')
engine = sa.create_engine(conn_str)
engine

In [ ]:
query_labs = """

with cmb_meas_persons as
(
select * from trg_analytics.fact_lab_observations
where (('80053'=ANY(test_codes)) or
        ('80048'=ANY(test_codes)) or
        ('1188'=ANY(test_codes)) or
        ('85027'=ANY(test_codes)) or
         ('82948'=ANY(test_codes)) or
       ('85025'=ANY(test_codes)) ) 

and test_date between '2017-01-01' and '2017-06-30'
)

select distinct on (personid, result_description) *
from cmb_meas_persons
where result_description is not null
order by personid, result_description, test_date desc
 
"""

print(query_labs)

with engine.begin() as conn:
    df_labs = pd.read_sql(sa.text(query_labs), conn)

In [ ]:
query_hosps = """
select personid, sum(case when (contact_type = 'hospital'
and contact_subtype = 'admission') then 1 else 0 end) as hosp_visits,
sum(case when (contact_type = 'hospital'
and contact_subtype = 'emergency_department') then 1 else 0 end) as er_visits,
sum(case when (contact_type = 'hospital'
and contact_subtype = 'observation') then 1 else 0 end) as obs_visits
from trg_analytics.fact_contact_events 
where low_service_date between '2017-07-01' and '2017-12-31'
group by personid
"""

print(query_hosps)

with engine.begin() as conn:
    df_hosps = pd.read_sql(sa.text(query_hosps), conn)


In [ ]:
query_hosps_pred = """



select personid, sum(case when (contact_type = 'hospital'
and contact_subtype = 'admission') then 1 else 0 end) as hosp_visits_prev,
sum(case when (contact_type = 'hospital'
and contact_subtype = 'emergency_department') then 1 else 0 end) as er_visits_prev,
sum(case when (contact_type = 'hospital'
and contact_subtype = 'observation') then 1 else 0 end) as obs_visits_prev
from trg_analytics.fact_contact_events 
where low_service_date between '2017-01-01' and '2017-06-30'
group by personid

"""

print(query_hosps_pred)

with engine.begin() as conn:
    df_hosps_pred = pd.read_sql(sa.text(query_hosps_pred), conn)

## Process and Clean Data

In [ ]:
df_labs=df_labs[df_labs.result_value!='ALT. TEST PERFORMED']
df_labs=df_labs[df_labs.result_value!='SEE NOTE']
df_labs=df_labs[df_labs.result_value!='NOT MEASURED']
df_labs=df_labs[df_labs.result_value!='false']

In [ ]:
df_labs=df_labs.loc[df_labs.result_description!='glomerular_filtration_rate_1_73_sq_m_predicted_among_blacks_by_creatinine_based_formula_(mdrd)',:]
df_labs=df_labs.loc[df_labs.result_description!='glomerular_filtration_rate_1_73_sq_m_predicted_by_creatinine_based_formula_(mdrd)',:]
df_labs=df_labs.loc[df_labs.result_description!='glomerular_filtration_rate_1_73_sq_m_predicted_by_creatinine_based_formula_(ckd_epi)',:]





In [ ]:
df_labs.result_value=pd.to_numeric(df_labs.result_value)

In [ ]:
# Process result_description to replace troublesome characters with '_'

temp = df_labs.result_description
df_labs.result_description = list(map(lambda x: x.replace('/','_').replace('[','_').replace(']','_').replace('<','_').replace('>','_')
                           .replace('.','_').replace(' ','_').replace('-','_').replace(',','_').lower(),temp))

# Create a short version of the name 
# This assumes, perhaps not safely, that if lab names have the same first x charaters, they are the same lab
df_labs['result_desc_short'] = np.array(list(map(lambda x: x[:15].lower(), df_labs.result_description)))

In [ ]:
df_labs.head(10)

#### Make a pivot table to get lab values into columns

In [ ]:
df_lab = df_labs.pivot_table(index='personid', values='result_value', columns = 'result_description').reset_index()
df_lab.head()

In [ ]:
df_lab.head()

In [ ]:
df_lab_short = df_labs.pivot_table(index='personid', values='result_value', columns = 'result_desc_short').reset_index()
df_lab_short.head()

In [ ]:
df_hosps.head(10)

#### Do some sanity checks on the data

In [ ]:
np.sum(np.isnan(df_hosps))

In [ ]:
np.sum(np.isnan(df_hosps_pred))

In [ ]:
df_full = pd.merge(df_lab_short, df_hosps_pred, how='left', on='personid')
#df_full.rename(columns = {'hosp_visits':'hosp_visits_prev'}, inplace = True)

df_full = pd.merge(df_full, df_hosps, how='left', on='personid')

In [ ]:
np.sum(np.isnan(df_full))

In [ ]:
## Fill in zeros for missing values in these columns

for col in ['hosp_visits_prev','er_visits_prev','obs_visits_prev','hosp_visits','er_visits','obs_visits']:
    df_full[col] = np.nan_to_num(df_full[col])
np.sum(np.isnan(df_full))

In [ ]:
df_full['is_utilizer'] = ((df_full.hosp_visits>0) | (df_full.er_visits>0)).astype(int)

In [ ]:
## Define some sets of variables

selected_predictors_labs = [
#'hosp_visits_prev',
#'er_visits_prev',
#'obs_visits_prev',
'glucose__mass_v',
'glomerular_filt',
'erythrocytes__#',  
'platelets__#_vo',
'leukocytes__#_v', 
'hematocrit__vol', 
'urea_nitrogen_c', 
'hemoglobin__mas']

selected_predictors_ut = [
'hosp_visits_prev',
'er_visits_prev',
'obs_visits_prev',
]

selected_predictors_utlabs = [
'hosp_visits_prev',
'er_visits_prev',
'obs_visits_prev',
'glucose__mass_v',
'glomerular_filt',
'erythrocytes__#',  
'platelets__#_vo',
'leukocytes__#_v', 
'hematocrit__vol', 
'urea_nitrogen_c', 
'hemoglobin__mas']



selected_predictors_big = [
'er_visits_prev',
'hosp_visits_prev',
'obs_visits_prev',
'erythrocyte_dis',
'glomerular_filt',
'erythrocyte_mea', 
'erythrocytes__#', 
'lymphocytes_100',
'glucose__mass_v',
'leukocytes__#_v',
'eosinophils_100', 
'platelets__#_vo',
'alkaline_phosph',
'neutrophils_100',
'urea_nitrogen_c',
'urea_nitrogen__',
'neutrophils__#_',
'platelet_mean_v',
'hemoglobin__mas',
'hematocrit__vol',
'glucose_random',
'basophils_100_l',
'monocytes_100_l',
'calcium__mass_v',
'albumin__mass_v',
'lymphocytes__#_',
'alanine_aminotr',
'aspartate_amino',
'potassium__mole',
'neutrophils_ban',
'sodium__moles_v',
'creatinine__mas',
'monocytes__#_vo',
'globulin__mass_',
'bicarbonate__mo',
'bilirubin_total',
'eosinophils__#_',
'hematocrit',
'protein__mass_v',
'mononuclear_cel',
'chloride__moles',
'albumin_globuli']

selected_predictors = selected_predictors_ut

### Define some useful functions

In [ ]:
def plot_reliability_diagram(y,x,bins=np.linspace(0,1,21),size_points=True, show_baseline=True,
                                error_bars=True, ax=None, marker='+',c='red', **kwargs):
    if ax is None:
        ax = plt.gca()
        fig = ax.get_figure()
    digitized_x = np.digitize(x, bins)
    mean_count_array = np.array([[np.mean(y[digitized_x == i]),len(y[digitized_x == i]),np.mean(x[digitized_x==i])] for i in np.unique(digitized_x)])
    x_pts_to_graph = mean_count_array[:,2]
    y_pts_to_graph = mean_count_array[:,0]
    pt_sizes = mean_count_array[:,1]
    if show_baseline:
        ax.plot(np.linspace(0,1,100),(np.linspace(0,1,100)),'k--')
    for i in range(len(y_pts_to_graph)):
        if size_points:
            plt.scatter(x_pts_to_graph,y_pts_to_graph,s=pt_sizes,marker=marker,c=c, **kwargs)
        else:
            plt.scatter(x_pts_to_graph,y_pts_to_graph, **kwargs)
    #plt.axis([-0.1,1.1,-0.1,1.1])
    if error_bars:
        plt.errorbar(x_pts_to_graph, x_pts_to_graph,
                    yerr=1.96*x_pts_to_graph*(1-x_pts_to_graph)/(np.sqrt(pt_sizes)), capsize=5)

    return(x_pts_to_graph,y_pts_to_graph,pt_sizes)

In [ ]:
def histogram_pair(value_vec, binary_vec, **kwargs):
    plt.figure(figsize = (10,6))
    plt.subplot(3,1,1)
    out1 = plt.hist(value_vec[binary_vec==1], **kwargs)
    plt.subplot(3,1,2)
    out0 = plt.hist(value_vec[binary_vec==0], **kwargs)
    bin_leftpts = (out1[1])[:-1]
    bin_rightpts = (out1[1])[1:]
    bin_centers = (bin_leftpts+bin_rightpts)/2
    prob_numer = out1[0]
    prob_denom = out1[0]+out0[0]
    smoothing_const = .001
    probs = (prob_numer+smoothing_const)/(prob_denom+2*smoothing_const)
    plt.subplot(3,1,3)
    plt.plot(bin_centers, prob_numer/prob_denom)
    plt.errorbar(bin_centers,probs, yerr=1.96*probs*(1-probs)/np.sqrt(prob_denom),capsize=3 )
    plt.xlim(bin_leftpts[0], bin_rightpts[-1])
    

In [ ]:
def train_test_evaluate_model(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr,y_tr)
    test_preds = model.predict_proba(X_te)[:,1]
    prec, rec, _ = precision_recall_curve(y_test, test_preds)
    fpr, tpr, _ = roc_curve(y_test, test_preds)
    
    plt.figure(figsize=(20,8))
    plt.subplot(1,3,1)
    plt.plot(prec[:-1], rec[:-1])
    plt.xlim(0, 1) 
    plt.ylim(0,1)

    plt.subplot(1,3,2)
    plt.plot(fpr, tpr)
    plt.xlim(0, 1) 
    plt.ylim(0,1)
    plt.plot([0,1],[0,1], 'k--')

    plt.subplot(1,3,3)
    plot_reliability_diagram(y_test,test_preds)
    
    return roc_auc_score(y_test, test_preds), log_loss(y_test, test_preds)

### Do a little data exploration

In [ ]:
histogram_pair(df_full['hosp_visits_prev'].values, df_full.is_utilizer.values, bins=10, range = (-.5,9.5))

In [ ]:
histogram_pair(df_full['er_visits_prev'].values, df_full.is_utilizer, bins=10, range = (-.5,9.5))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score, log_loss, roc_curve

In [ ]:
X = df_full.loc[:,selected_predictors]
y = df_full['is_utilizer']
#y = ((df_full.hosp_visits>0) | (df_full.er_visits>0) | (df_full.obs_visits>0)).astype(int)


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .25, random_state=42)
X_train.shape, X_test.shape

In [ ]:
y_train.value_counts(), y_test.value_counts(), y_train.mean(), y_test.mean()

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

### Logistic Regression "off-the-shelf"
In sklearn, the "default" Logistic Regression is actually an L2-Penalized Logistic Regresssion with C=1 

In [ ]:
lr_model0 = LogisticRegression()  
train_test_evaluate_model(lr_model0, X_train, y_train, X_test, y_test)

In [ ]:
list(zip(selected_predictors,lr_model0.coef_[0]))

In [ ]:
lr_model0a = LogisticRegression(C=1000000)  
train_test_evaluate_model(lr_model0a, X_train, y_train, X_test, y_test)

In [ ]:
list(zip(selected_predictors,lr_model0a.coef_[0]))

In [ ]:
lr_model0b = LogisticRegression(C=.01)  
train_test_evaluate_model(lr_model0b, X_train, y_train, X_test, y_test)

In [ ]:
list(zip(selected_predictors,lr_model0b.coef_[0]))

### Using Cross-Validation to choose best Nuisance Parameter Value
Rather than just picking arbitrary values of "C", let's use cross-validation to try and find the best value.  Also, let's use an L1 penalty rather than L2, so that we get a more parsimonious model.

In [ ]:
from sklearn.metrics import make_scorer

In [ ]:
lr_model1 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear', 
                                 scoring = make_scorer(log_loss, greater_is_better=False))  
train_test_evaluate_model(lr_model1, X_train, y_train, X_test, y_test)

In [ ]:
list(zip(selected_predictors,lr_model1.coef_[0]))

In [ ]:
lr_model1.Cs_, lr_model1.C_

In [ ]:
np.mean(lr_model1.scores_[1], axis=0)

In [ ]:
plt.plot(np.log10(lr_model1.Cs_),np.mean(lr_model1.scores_[1], axis=0), 'x-')

### Standardizing Variables Beforehand
The issue with regularizing based on coefficient size is that arbitrary differences in the values of the variables (such as units) can have a big effect on how a variable is penalized.  One solution is to "standardize" the variables beforehand (i.e. subtract the mean and divide by the standard deviation of the variable).

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

In [ ]:
std_scaler.fit(X_train)

In [ ]:
std_scaler.mean_ ,std_scaler.scale_

In [ ]:
X_train_std  = std_scaler.transform(X_train)
X_test_std = std_scaler.transform(X_test)

In [ ]:
lr_model2 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear',
                                scoring = make_scorer(log_loss, greater_is_better=False))    
train_test_evaluate_model(lr_model2, X_train_std, y_train, X_test_std, y_test)

In [ ]:
list(zip(selected_predictors,lr_model2.coef_[0]))

### Repeat this process using just labs

In [ ]:
selected_predictors = selected_predictors_labs

X = df_full.loc[:,selected_predictors]
y = ((df_full.hosp_visits>0) | (df_full.er_visits>0)).astype(int)
#y = ((df_full.hosp_visits>0) | (df_full.er_visits>0) | (df_full.obs_visits>0)).astype(int)
plot_mat = X.copy()
plot_mat["had_hosp_visit"] = y


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .25, random_state=42)
X_train.shape, X_test.shape

#### Need to remove missing values - will impute the median

In [ ]:
from sklearn.preprocessing import Imputer
impute_median = Imputer(strategy = 'median')
X_train_med = impute_median.fit_transform(X_train)
X_test_med = impute_median.transform(X_test)

In [ ]:
X_train_med = pd.DataFrame(X_train_med, columns = selected_predictors)
X_test_med = pd.DataFrame(X_test_med, columns = selected_predictors)

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(X_train_med)
X_train_med_std  = std_scaler.transform(X_train_med)
X_test_med_std = std_scaler.transform(X_test_med)

In [ ]:
lr_model0 = LogisticRegression()
train_test_evaluate_model(lr_model0, X_train_med, y_train, X_test_med, y_test)

In [ ]:
list(zip(selected_predictors,lr_model0.coef_[0]))

In [ ]:
lr_model1 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear', 
                                 scoring = make_scorer(log_loss, greater_is_better=False))  
train_test_evaluate_model(lr_model1, X_train_med, y_train, X_test_med, y_test)

In [ ]:
list(zip(selected_predictors,lr_model1.coef_[0]))

In [ ]:
lr_model2 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear', 
                                 scoring = make_scorer(log_loss, greater_is_better=False))  
train_test_evaluate_model(lr_model2, X_train_med_std, y_train, X_test_med_std, y_test)

In [ ]:
list(zip(selected_predictors,lr_model2.coef_[0]))

In [ ]:
train_test_evaluate_model(lr_model2, X_train_med_std, y_train, X_test_med_std, y_test)

In [ ]:
list(zip(selected_predictors,lr_model2.coef_[0]))

In [ ]:
histogram_pair(X_train_med['erythrocytes__#'].values,y_train.values, bins=12, range = (2,8))

In [ ]:
histogram_pair(X_train_med['hematocrit__vol'].values,y_train.values, bins=24, range=(28,52))

In [ ]:
histogram_pair(X_train_med['platelets__#_vo'].values,y_train.values,bins=np.concatenate(([0,100],np.linspace(150,300,16),[350,450,600])))

### Repeat with both labs and utilization

In [ ]:
selected_predictors = selected_predictors_utlabs

X = df_full.loc[:,selected_predictors]
y = ((df_full.hosp_visits>0) | (df_full.er_visits>0)).astype(int)
#y = ((df_full.hosp_visits>0) | (df_full.er_visits>0) | (df_full.obs_visits>0)).astype(int)
plot_mat = X.copy()
plot_mat["had_hosp_visit"] = y


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .25, random_state=42)
X_train.shape, X_test.shape

In [ ]:
from sklearn.preprocessing import Imputer
impute_median = Imputer(strategy = 'median')
X_train_med = impute_median.fit_transform(X_train)
X_test_med = impute_median.transform(X_test)

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(X_train_med)
X_train_med_std  = std_scaler.transform(X_train_med)
X_test_med_std = std_scaler.transform(X_test_med)

In [ ]:
lr_model0 = LogisticRegression()
train_test_evaluate_model(lr_model0, X_train_med, y_train, X_test_med, y_test)

In [ ]:
list(zip(selected_predictors,lr_model0.coef_[0]))

In [ ]:
lr_model1 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear', 
                                 scoring = make_scorer(log_loss, greater_is_better=False))  
train_test_evaluate_model(lr_model1, X_train_med, y_train, X_test_med, y_test)

In [ ]:
list(zip(selected_predictors,lr_model1.coef_[0]))

In [ ]:
lr_model2 = LogisticRegressionCV(Cs = 10**np.linspace(-5,5,61), penalty='l1', solver = 'liblinear', 
                                 scoring = make_scorer(log_loss, greater_is_better=False))  
train_test_evaluate_model(lr_model2, X_train_med_std, y_train, X_test_med_std, y_test)

In [ ]:
list(zip(selected_predictors,lr_model2.coef_[0]))